In [1]:
pip install SQLAlchemy==1.4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.17-cp310-cp310-linux_x86_64.whl size=1519928 sha256=b752405eac624c03e58ff83cbcf8ebfa1efdd8c4b518778f73dfe6e9f4e05a3f
  Stored in directory: /root/.cache/pip/wheels/eb/d0/1e/ab0fcbac6f74a69b788a37fccd365fc67535b5ab79f97bd860
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.17 which is incompatible.


In [2]:
pip install pymssql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 30.7 MB/s eta 0:00:00


In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.6 MB/s eta 0:00:00


In [4]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import statsmodels.formula.api as smf
from pandas_datareader import DataReader as pdr

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor as rf
from catboost import CatBoostRegressor as sbr

In [5]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912"
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

## Strategy 2

In [6]:
df = pd.read_sql(
    """
    select date, ticker, mom1m, mom12m, mom36m, siccd, ret, sfe, rsup, ear, chmom, idiovol
    from data
    order by date, ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index(["date", "ticker"])
features = ["mom12m", "sfe", "rsup", "ear", "chmom", "idiovol"]

In [7]:
df["actual"] = df.ret

qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)
df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d:
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )
)

In [8]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes = pd.DataFrame(codes).reset_index()
codes.columns = ["siccd", "industry"]

df = df.reset_index().merge(codes, on="siccd")
df = df.set_index(["date", "ticker"])

features.append("industry")

Pipeline 1: Also favourite pipeline

quantile transform + column transform + polynomial transform + model

In [9]:
pipe1 = sbr(cat_features=["industry"], random_state=42)


Looping test

## Strategy 2

In [10]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
predictions = None
train_score = []
test_score = []

for train_date, end_date in zip(dates[:-1], dates[1:]):

  fltr1 = df.index.get_level_values("date") < train_date
  fltr2 = df.index.get_level_values("date") < end_date
  train = df[fltr1]
  test = df[~fltr1 & fltr2]

  Xtrain = train[features]
  ytrain = train["ret"]
  Xtest = test[features]
  ytest = test["ret"]

  pipe1.fit(Xtrain, ytrain)
  print('Train set score: ' + str(pipe1.score(Xtrain, ytrain)))


  pred = pipe1.predict(Xtest)
  pred = pd.Series(pred, index=test.index)
  predictions = pd.concat((predictions, pred))
  print('Test set score: ' + str(pipe1.score(Xtest,ytest)))

df["predict"] = predictions

Learning rate set to 0.091534
0:	learn: 1.0063001	total: 135ms	remaining: 2m 14s
1:	learn: 1.0059118	total: 183ms	remaining: 1m 31s
2:	learn: 1.0055026	total: 238ms	remaining: 1m 18s
3:	learn: 1.0052034	total: 299ms	remaining: 1m 14s
4:	learn: 1.0049879	total: 347ms	remaining: 1m 8s
5:	learn: 1.0047761	total: 393ms	remaining: 1m 5s
6:	learn: 1.0045852	total: 457ms	remaining: 1m 4s
7:	learn: 1.0043558	total: 510ms	remaining: 1m 3s
8:	learn: 1.0041644	total: 568ms	remaining: 1m 2s
9:	learn: 1.0040412	total: 610ms	remaining: 1m
10:	learn: 1.0038738	total: 656ms	remaining: 59s
11:	learn: 1.0037336	total: 720ms	remaining: 59.3s
12:	learn: 1.0036473	total: 775ms	remaining: 58.9s
13:	learn: 1.0035230	total: 824ms	remaining: 58s
14:	learn: 1.0034193	total: 886ms	remaining: 58.2s
15:	learn: 1.0033128	total: 935ms	remaining: 57.5s
16:	learn: 1.0032111	total: 989ms	remaining: 57.2s
17:	learn: 1.0031015	total: 1.03s	remaining: 56.5s
18:	learn: 1.0030317	total: 1.08s	remaining: 55.8s
19:	learn: 1.0

In [11]:
df

mom1m    mom12m    mom36m  siccd       ret       sfe  \
date    ticker                                                            
2000-01 AA      0.267176  1.171749  0.207338   3334 -1.067024  0.465527   
        KLU     0.160377  0.802712 -0.580645   3334 -1.552878 -1.531518   
2000-02 AA     -0.160392  1.258388  0.250118   3334 -0.024881  1.103803   
        KLU    -0.219512  0.866253 -0.601942   3334 -0.315010 -1.529685   
2000-03 AA     -0.013453  0.936004  0.175634   3334 -0.026778  0.869391   
...                  ...       ...       ...    ...       ...       ...   
2021-11 KSPN   -0.129874  0.172230 -0.426020   5735 -0.316798 -1.154158   
2021-12 KSPN   -0.062392 -0.333494 -0.346592   5735 -1.866321 -1.149192   
2022-01 KSPN   -0.232353 -0.651999  0.161111   5735 -1.916238 -1.185322   
2022-02 KSPN   -0.281609 -2.058952  2.295728   5735  0.556408 -1.166941   
2022-03 KSPN    0.047333 -2.059576  2.393618   5735 -0.834250 -1.117105   

                    rsup       ear     chmom   idiovol    actual  \
date    ticker                                                     
2000-01 AA      0.261592 -0.063497 -0.619031 -0.438334 -0.160392   
        KLU    -1.692193 -1.158489 -1.731264  0.373475 -0.219512   
2000-02 AA      0.247164 -0.037780 -0.486276 -0.451970 -0.013453   
        KLU    -1.703658 -1.149544 -1.951912  0.373893 -0.052083   
2000-03 AA     -0.711993 -0.307048 -1.138947 -0.444358  0.025547   
...                  ...       ...       ...       ...       ...   
2021-11 KSPN    1.813806 -0.284599 -1.322762  1.962924 -0.062392   
2021-12 KSPN    1.443499 -0.436823 -1.613696  1.951665 -0.232353   
2022-01 KSPN   -2.378046 -0.949205 -1.652350  1.966406 -0.281609   
2022-02 KSPN   -2.330739 -0.930033  0.152451  1.994387  0.047333   
2022-03 KSPN   -2.203424 -0.821604 -0.550940  2.013429 -0.075646   

                     industry   predict  
date    ticker                           
2000-01 AA      Manufacturing       NaN  
        KLU     Manufacturing       NaN  
2000-02 AA      Manufacturing       NaN  
        KLU     Manufacturing       NaN  
2000-03 AA      Manufacturing       NaN  
...                       ...       ...  
2021-11 KSPN            Shops -0.106398  
2021-12 KSPN            Shops -0.022576  
2022-01 KSPN            Shops -0.072447  
2022-02 KSPN            Shops -0.155946  
2022-03 KSPN            Shops -0.054670  

[670388 rows x 13 columns]

In [12]:
df.to_csv('strat2.csv')

In [13]:
numlong = 150
numshort = 100

df["rank_from_top"] = df.groupby("date").predict.rank(
  method="first",
  ascending=False
)
df["long"] = df.rank_from_top <= numlong

df["rank_from_bottom"] = df.groupby("date").predict.rank(
  method="first"
)
df["short"] = df.rank_from_bottom <= numshort

In [14]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
df = df[df.index.get_level_values("date") >= dates[0]]

long_ret = df[df.long].groupby("date").actual.mean()
short_ret = df[df.short].groupby("date").actual.mean()

In [15]:
ret = 1.5*long_ret - 0.5*short_ret
ret

date
2005-01   -0.023780
2005-02    0.028594
2005-03   -0.038029
2005-04   -0.061342
2005-05    0.110222
             ...   
2021-11   -0.006928
2021-12    0.134046
2022-01   -0.025880
2022-02    0.037494
2022-03   -0.008025
Name: actual, Length: 207, dtype: float64

In [16]:
df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
df_new.index = df_new.index.astype(str)
df_new["ret"] = ret
df_new["ret_rf"] = df_new.ret - df_new.RF
df_new = df_new.dropna()
df_new.index = df_new.index.astype(str)
df_new.index.name = "date"
df_new = df_new.reset_index()
df_new = df_new.rename(columns={"Mkt-RF": "mkt_rf", "RF": "rf"})

<ipython-input-16-c9ad2a6d9b27>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
<ipython-input-16-c9ad2a6d9b27>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100


In [17]:
#Computing alpha
result = smf.ols("ret_rf~mkt_rf+SMB+HML+CMA+RMW", df_new).fit()
betas = result.params[1:]
mkt = df_new.rf + betas[0]*df_new.mkt_rf
smb = betas[1]*df_new.SMB
hml = betas[2]*df_new.HML
cma = betas[3]*df_new.CMA
rmw = betas[4]*df_new.RMW
active = df_new.ret - mkt - smb - hml - cma - rmw


In [18]:
print("Active is found to be", active)

Active is found to be 0     -0.001907
1      0.001939
2     -0.021979
3     -0.018387
4      0.060800
         ...   
202    0.002357
203    0.081590
204    0.007148
205    0.037021
206   -0.023059
Length: 207, dtype: float64


In [19]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 ret_rf   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     100.2
Date:                Thu, 02 May 2024   Prob (F-statistic):           1.18e-52
Time:                        22:24:16   Log-Likelihood:                 410.08
No. Observations:                 207   AIC:                            -808.2
Df Residuals:                     201   BIC:                            -788.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0076      0.002      3.059      0.0